#2.0

[New paper](https://www.ripublication.com/ijaer19/ijaerv14n8_33.pdf)

In [ ]:
import os

import tensorflow as tf
import keras
import argparse
from myconfig import configt
from keras import optimizers
from keras.utils import np_utils
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

#conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(230, 230, 3))
#from keras.applications.xception import Xception
#conv_base = Xception(weights='imagenet', include_top=False, input_shape=(96, 96, 3))


from keras import models
from keras import layers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import numpy as np
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Average
from keras.layers import merge

from keras import applications
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Model, Input

import argparse

from keras import optimizers
import seaborn as sns
#from keras.engine.topology import Input

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--plot", type=str, default="plot.png",
	help="path to output loss/accuracy plot")
args = vars(ap.parse_args())

# initialize our number of epochs, initial learning rate, and batch
# size


chanDim=-1

def resnet500(model_input):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    last = base_model.output
    x = layers.Flatten()(last)
    x = Dense(256, activation='relu')(x)
    #preds = Dense(8, activation='softmax')(x)
    #model = Model(base_model.input, preds)
    model = Model(base_model.input, x)

    base_model.trainable = True
    set_trainable = False
    for layer in base_model.layers:
        if layer.name == 'add_30': # 从这一层开始往后均可训练
            set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False

    return model

def vgg166(model_input):
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=model_input)
    last = base_model.output
    x = layers.Flatten()(last)
    x = Dense(256, activation='relu')(x)
    #preds = Dense(8, activation='softmax')(x)
    model = Model(base_model.input, x)

    base_model.trainable = True
    set_trainable = False
    for layer in base_model.layers:
        if layer.name == 'block5_conv1': # 从这一层开始往后均可训练
            set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False

    return model

model_input = Input(shape=(230, 230, 3))


model1 = resnet500(model_input)
model2 = vgg166(model_input)

ensembled_models = [model1,model2]
def ensemble(models,model_input):
    outputs = [model.outputs[0] for model in models]
    #modelo1 = model1.outputs[0]
    #modelo2 = model2.outputs[0]
    modelo1 = outputs[0]
    modelo2 = outputs[1]
    y = layers.concatenate([modelo1, modelo2], axis=-1)
    preds = Dense(8, activation='softmax')(y)
    model = Model(model_input,preds,name='ensemble')
    return model


ensemble_model = ensemble(ensembled_models,model_input)


from keras.utils import plot_model
plot_model(ensemble_model, show_shapes=True, to_file='bcdnet_comodel.png')
#added = keras.layers.add([input1, input2])


# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--plot", type=str, default="plot.png",
	help="path to output loss/accuracy plot")
args = vars(ap.parse_args())

# initialize our number of epochs, initial learning rate, and batch
# size
NUM_EPOCHS = 38
INIT_LR = 1e-2
BS = 16

# determine the total number of image paths in training, validation,
# and testing directories
trainPaths = list(paths.list_images(configt.TRAIN_PATH))
totalTrain = len(trainPaths)
totalVal = len(list(paths.list_images(configt.VAL_PATH)))
totalTest = len(list(paths.list_images(configt.TEST_PATH)))

# account for skew in the labeled data
trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels = np_utils.to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)
classWeight = classTotals.max() / classTotals

# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	configt.TRAIN_PATH,
	class_mode="categorical",
	target_size=(230, 230),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	configt.VAL_PATH,
	class_mode="categorical",
	target_size=(230, 230),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS )

# initialize the testing generator
testGen = valAug.flow_from_directory(
	configt.TEST_PATH,
	class_mode="categorical",
	target_size=(230, 230),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

# initialize our CancerNet model and compile it

#opt = Adagrad(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
#opt= Ranger(params=CancerNet.parameters(), lr=1e-3, alpha=0.5, k=6, N_sma_threshhold=5, betas=(.95,0.999), eps=1e-5, weight_decay=0)
#opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#opt = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=0.5, decay=INIT_LR / NUM_EPOCHS)
#opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0)
#opt = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=0.05, decay=1e-2)
#opt = optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
ensemble_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])



# 绘制训练过程中的损失曲线和精度曲线




H = ensemble_model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)

# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = ensemble_model.predict_generator(testGen,
	steps=(totalTest // BS) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
# label with corresponding largest predicted probability
ensemble_model.save("zuhe.h5")
predIdxs = np.argmax(predIdxs, axis=1)
print(confusion_matrix(testGen.classes, predIdxs))
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys(), digits=4))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity


import matplotlib.pyplot as plt

acc = H.history['acc']
val_acc = H.history['val_acc']
loss = H.history['loss']
val_loss = H.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b:', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.plot(epochs, loss, 'b--', label='Training loss')
plt.plot(epochs, val_loss, 'r-.', label='Validation loss')
plt.title('Training Loss and Accuracy on Dataset')
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()




# === 混淆矩阵：真实值与预测值的对比 ===
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
con_mat = confusion_matrix(testGen.classes, predIdxs)

con_mat_norm = con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis]     # 归一化

con_mat_norm = np.around(con_mat_norm, decimals=4)

# === plot ===
plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_norm, annot=True, cmap='Blues')

plt.ylim(0, 8)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')


plt.show()






#2.1 pytorch
[paper new final](https://www.techscience.com/ueditor/files/cmes/TSP_CMES-130-2/TSP_CMES_17030/TSP_CMES_17030.pdf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.data import Subset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.optim.lr_scheduler import ExponentialLR, StepLR, ReduceLROnPlateau

import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)



# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

PyTorch Version:  1.10.0+cu111
Torchvision Version:  0.11.1+cu111


In [ ]:
TRAIN_DIR =  '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_train/' #where train total : 1600 (400 per folder)
VAL_DIR =  '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_val/' #where val total : 80 (20 per folder)

In [ ]:
data_transform1 = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()])
        #transforms.Normalize(mean=[0.5, 0.5, 0.5],     std=[0.5, 0.5, 0.5]) # transforms.Normalize(mean=[0.8513, 0.6445, 0.8938],std=[0.1047, 0.1634, 0.0739]) #


data_transform2 = transforms.Compose([
        transforms.Resize(224),
        #transforms.RandomRotation(90),
        #transforms.RandomHorizontalFlip(0.8),
        transforms.ToTensor()])
        #transforms.Normalize(mean=[0.5, 0.5, 0.5],     std=[0.5, 0.5, 0.5]) #transforms.Normalize(mean=[0.8513, 0.6445, 0.8938],  std=[0.1047, 0.1634, 0.0739])


train_orig = torchvision.datasets.ImageFolder(TRAIN_DIR,
                                           transform=data_transform1)

#train_aug = torchvision.datasets.ImageFolder(TRAIN_DIR,
 #                                          transform=data_transform2)


#increased_train = torch.utils.data.ConcatDataset([train_aug,train_orig])

In [ ]:
len(train_orig)

480

In [ ]:
val_orig = torchvision.datasets.ImageFolder(VAL_DIR,
                                           transform=data_transform1)

#val_aug = torchvision.datasets.ImageFolder(VAL_DIR,
 #                                          transform=data_transform2)


#increased_val = torch.utils.data.ConcatDataset([val_aug,val_orig])

In [ ]:
len(val_orig)

40

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 8

# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train
num_epochs = 80

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            #scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(
                        nn.Linear(num_ftrs, 256),
                        nn.ReLU(),
                        #nn.Dropout(0.4),                             #dropout here
                        nn.Linear(256, num_classes),
                        nn.Softmax(dim=1))
        #model_ft.fc = nn.Dropout(0.4)
        #model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        #model_ft.classifier = nn.Sequential(
         #               nn.Linear(num_ftrs, 256),
          #              nn.ReLU(),
           #             nn.Dropout(0.4),                          #dropout here
            #            nn.Linear(256, num_classes))
        #model_ft.classifier = nn.Dropout(0.3)
        #model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch.utils.data as data
from torch.autograd import Variable
import numpy as np


train_loader = data.DataLoader(train_orig, batch_size=batch_size, shuffle=True,  num_workers=4)
val_loader  = data.DataLoader(val_orig, batch_size=batch_size, shuffle=True, num_workers=4)


dataloaders_dict = {'train':train_loader, 'val':val_loader}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 fc.0.weight
	 fc.0.bias
	 fc.2.weight
	 fc.2.bias


In [ ]:
#scheduler2 = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

optimizer_ft = optim.Adam(params_to_update, lr=0.00001, betas=(0.9, 0.999), eps=1e-08, amsgrad=False)#optim.SGD(params_to_update, lr=0.001, momentum=0.9, weight_decay=1e-2) #
#schedulr = ExponentialLR(optimizer_ft, gamma=1)#StepLR(optimizer_ft, step_size=20, gamma=1) #ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.1, patience=2, threshold=0.0001)#


In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/79
----------
train Loss: 2.0788 Acc: 0.1167
val Loss: 2.0797 Acc: 0.1250

Epoch 1/79
----------
train Loss: 2.0760 Acc: 0.1750
val Loss: 2.0799 Acc: 0.1250

Epoch 2/79
----------
train Loss: 2.0738 Acc: 0.2375
val Loss: 2.0789 Acc: 0.0750

Epoch 3/79
----------
train Loss: 2.0717 Acc: 0.2729
val Loss: 2.0777 Acc: 0.1250

Epoch 4/79
----------
train Loss: 2.0695 Acc: 0.2542
val Loss: 2.0768 Acc: 0.1500

Epoch 5/79
----------
train Loss: 2.0668 Acc: 0.2583
val Loss: 2.0760 Acc: 0.1250

Epoch 6/79
----------
train Loss: 2.0646 Acc: 0.2313
val Loss: 2.0755 Acc: 0.1000

Epoch 7/79
----------
train Loss: 2.0617 Acc: 0.2313
val Loss: 2.0743 Acc: 0.1500

Epoch 8/79
----------
train Loss: 2.0592 Acc: 0.2542
val Loss: 2.0736 Acc: 0.1250

Epoch 9/79
----------
train Loss: 2.0572 Acc: 0.2271
val Loss: 2.0723 Acc: 0.1250

Epoch 10/79
----------
train Loss: 2.0542 Acc: 0.2521
val Loss: 2.0713 Acc: 0.1250

Epoch 11/79
----------
train Loss: 2.0501 Acc: 0.2646
val Loss: 2.0698 Acc: 0.1250

Ep

In [ ]:
MODEL_DIR= '/content/drive/My Drive/FEW_SHOT_MULTI_TASK/Comparison/'
torch.save(model_ft, MODEL_DIR + 'Paper2_new_final_resnet50.pt')

In [ ]:
MODEL_DIR= '/content/drive/My Drive/FEW_SHOT_MULTI_TASK/Comparison/'

#model = models.resnet18(pretrained=True)
#num_ftrs = model.fc.in_features
#model.fc = nn.Linear(num_ftrs, 4)  # make the change

#model.load_state_dict(torch.load(MODEL_DIR + 'resnet_model.pt' ))
model = torch.load(MODEL_DIR + 'Paper2_new_final_resnet50.pt')
model.eval()
#device = 'cuda' if torch.cuda.is_available else 'cpu'

#model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:

TEST_DIR = '/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_test/'
test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()
        #transforms.Normalize(mean=[0.5, 0.5, 0.5],     std=[0.5, 0.5, 0.5])#         transforms.Normalize(mean=[0.8513, 0.6445, 0.8938],std=[0.1047, 0.1634, 0.0739])         #
    ])
test_ds = torchvision.datasets.ImageFolder(TEST_DIR,
                                           transform=test_transform)
test_loader = data.DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

In [ ]:
check_accuracy(test_loader, model)

Got 310 / 1876 with accuracy 16.52


In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            #num_correct += (predictions == y).sum()
            #num_samples += predictions.size(0)

        return predictions

    model.train()

In [ ]:
prd = check_accuracy(test_loader, model)

In [ ]:
prd.shape

torch.Size([4])

In [ ]:
train_features, test_labels = next(iter(test_loader))

In [ ]:
print(test_labels)

tensor([4, 3, 1, 5, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 5])


In [ ]:
test_labels.shape

torch.Size([16])

classification report

In [ ]:
from sklearn.metrics import roc_curve


#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(test_labels, prd)))

print('Micro Precision: {:.2f}'.format(precision_score(test_labels, prd, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(test_labels, prd, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(test_labels, prd, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(test_labels, prd, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(test_labels, prd, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(test_labels, prd, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(test_labels, prd, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(test_labels, prd, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(test_labels, prd, average='weighted')))


from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(test_labels, prd, target_names=['Class 0- A:', 'Class 1- F:', 'Class 2- PT:', 'Class 3- TA:','Class 4- DC:','Class 5- LC:','Class 6- MC:', 'Class 7- PC:'], digits = 4))


ValueError: ignored

AUC scores

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
#roc auc score
scr1 = roc_auc_score(test_labels, pred, multi_class='ovo', average='macro')
scr3 = roc_auc_score(test_labels, pred, multi_class='ovr', average='macro')
scr2 = roc_auc_score(test_labels, pred, multi_class='ovo', average='weighted')
scr4 = roc_auc_score(test_labels, pred, multi_class='ovr', average='weighted')

print("macro - ovo ROC AUC Score: ", scr1)
print("")

print("weighted - ovo ROC AUC Score: ", scr2)
print("")

print("macro - ovr ROC AUC Score: ", scr3)
print("")

print("weighted - ovr ROC AUC Score: ", scr4)
print("")

clss wise ROC curves and AUC scores

In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

target_names = ['A', 'F', 'PT', 'TA', 'DC','LC', 'MC', 'PC']
n_class = 8

for i in range(len(target_names)):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_labels, prd[:], pos_label=i)
    auroc = round(metrics.auc(fpr[i], tpr[i]),2)
    print('class',i,'-', target_names[i],' :','--AUC--->',auroc)

# plotting
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0-A: vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1- F: vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2- PT: vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--',color='yellow', label='Class 3- TA: vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='-',color='orange', label='Class 4-DC vs Rest')
plt.plot(fpr[5], tpr[5], linestyle='-',color='green', label='Class 5-LC: vs Rest')
plt.plot(fpr[6], tpr[6], linestyle='-',color='blue', label='Class 6-MC: vs Rest')
plt.plot(fpr[7], tpr[7], linestyle='-',color='yellow', label='Class 7- LC: vs Rest')


plt.title('Multiclass ROC-AUC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multilass ROC',dpi=300, bbox_inches='tight',   pad_inches = 0);


plt.show()

# final 2.1 keras
[paper new final](https://www.techscience.com/ueditor/files/cmes/TSP_CMES-130-2/TSP_CMES_17030/TSP_CMES_17030.pdf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

import tensorflow as tf
import keras
import argparse
#from myconfig import configt
from keras import optimizers
from keras.utils import np_utils
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

#conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(230, 230, 3))
#from keras.applications.xception import Xception
#conv_base = Xception(weights='imagenet', include_top=False, input_shape=(96, 96, 3))


from keras import models
from keras import layers
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import numpy as np
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Average
from keras.layers import merge

from keras import applications
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.models import Model, Input

import argparse

from keras import optimizers
import seaborn as sns
#from keras.engine.topology import Input


In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools


%matplotlib inline

In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale = 1./255)
       # rotation_range=40,
       # height_shift_range=0.2,
       # shear_range=0.2,
       # zoom_range=0.2,
       # horizontal_flip=True,
       # fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
trainGen = train_datagen.flow_from_directory(
    directory="/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=4,
    class_mode="categorical",
    shuffle=False,
    seed=42
)
valGen = valid_datagen.flow_from_directory(
    directory="/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_val/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=4,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 480 images belonging to 8 classes.
Found 40 images belonging to 8 classes.


In [ ]:
print("label map for train: ",trainGen.class_indices)
print("")
print("----")
print("")
print("label map for val data: ",valGen.class_indices)

label map for train:  {'benign_adenosis': 0, 'benign_fibroadenoma': 1, 'benign_phyllodestumor': 2, 'benign_tubularadenoma': 3, 'malignant_ductalcarcinoma': 4, 'malignant_lobularcarcinoma': 5, 'malignant_mucinouscarcinoma': 6, 'malignant_papillarycarcinoma': 7}

----

label map for val data:  {'benign_adenosis': 0, 'benign_fibroadenoma': 1, 'benign_phyllodestumor': 2, 'benign_tubularadenoma': 3, 'malignant_ductalcarcinoma': 4, 'malignant_lobularcarcinoma': 5, 'malignant_mucinouscarcinoma': 6, 'malignant_papillarycarcinoma': 7}


In [ ]:
print("Train classes: ",trainGen.classes)
print("")
print("Valid classes: ",valGen.classes)

Train classes:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]

Valid cla

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
testGen = test_datagen.flow_from_directory(
    directory="/content/drive/My Drive/BreaKHis2_8_trial/BreaKHis8_test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",  #None
    shuffle=False,
    #seed=42
)
nBatches = 1

Found 1876 images belonging to 8 classes.


In [ ]:
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
baseModel.summary()

[INFO] preparing model...
94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 


In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(8, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = True

In [ ]:
INIT_LR = 0.00001
BS = 16
NUM_EPOCHS = 80

In [ ]:
# compile the model
opt = Adam(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model
print("[INFO] training model...")
H = model.fit_generator(
	trainGen,
	validation_data=valGen,
	epochs=NUM_EPOCHS)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


[INFO] training model...
Epoch 1/80
120/120 [==============================] - 141s 1s/step - loss: 2.2090 - accuracy: 0.1167 - val_loss: 4.1446 - val_accuracy: 0.1000
Epoch 2/80
120/120 [==============================] - 21s 177ms/step - loss: 1.3645 - accuracy: 0.8687 - val_loss: 15.2018 - val_accuracy: 0.1250
Epoch 3/80
120/120 [==============================] - 21s 177ms/step - loss: 0.9082 - accuracy: 0.9979 - val_loss: 19.6710 - val_accuracy: 0.1250
Epoch 4/80
120/120 [==============================] - 21s 175ms/step - loss: 0.6065 - accuracy: 1.0000 - val_loss: 22.4243 - val_accuracy: 0.1250
Epoch 5/80
120/120 [==============================] - 21s 176ms/step - loss: 0.4192 - accuracy: 1.0000 - val_loss: 21.7071 - val_accuracy: 0.1250
Epoch 6/80
120/120 [==============================] - 21s 176ms/step - loss: 0.3023 - accuracy: 1.0000 - val_loss: 16.5292 - val_accuracy: 0.1250
Epoch 7/80
120/120 [==============================] - 21s 176ms/step - loss: 0.2280 - accuracy: 1.0000

In [ ]:
model.save("/content/drive/My Drive/FEW_SHOT_MULTI_TASK/Comparison/paper2_final_new_keras_retrain.h5")

In [ ]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen)
predIdxs = np.argmax(predIdxs, axis=1)
print(confusion_matrix(testGen.classes, predIdxs))
# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys(), digits=4))

[INFO] evaluating network...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


[[ 31  25   8  11   7  12   4   0]
 [117  36   9  12   5  35  23   1]
 [ 23  21  10   3   5  28   3   1]
 [ 55  34  10   5   1  18   7   2]
 [316 215  86  24   9 130  62  15]
 [ 44  55   1   8   2  18   5   7]
 [ 39  52  12  10   4  51  14   6]
 [ 47  10  26   4   1  34   4   3]]
                              precision    recall  f1-score   support

             benign_adenosis     0.0461    0.3163    0.0805        98
         benign_fibroadenoma     0.0804    0.1513    0.1050       238
       benign_phyllodestumor     0.0617    0.1064    0.0781        94
       benign_tubularadenoma     0.0649    0.0379    0.0478       132
   malignant_ductalcarcinoma     0.2647    0.0105    0.0202       857
  malignant_lobularcarcinoma     0.0552    0.1286    0.0773       140
 malignant_mucinouscarcinoma     0.1148    0.0745    0.0903       188
malignant_papillarycarcinoma     0.0857    0.0233    0.0366       129

                    accuracy                         0.0672      1876
                 

In [ ]:

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity


import matplotlib.pyplot as plt

acc = H.history['accuracy']
val_acc = H.history['val_accuracy']
loss = H.history['loss']
val_loss = H.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b:', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.plot(epochs, loss, 'b--', label='Training loss')
plt.plot(epochs, val_loss, 'r-.', label='Validation loss')
plt.title('Training Loss and Accuracy on Dataset')
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()




# === 混淆矩阵：真实值与预测值的对比 ===
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
con_mat = confusion_matrix(testGen.classes, predIdxs)

con_mat_norm = con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis]     # 归一化

con_mat_norm = np.around(con_mat_norm, decimals=4)

# === plot ===
plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_norm, annot=True, cmap='Blues')

plt.ylim(0, 8)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.savefig('Confusion matrix',dpi=300, bbox_inches='tight',   pad_inches = 0);

plt.show()

In [ ]:
preds = model.predict_generator(testGen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
#roc auc score
scr1 = roc_auc_score(testGen.classes, preds, multi_class='ovo', average='macro')
scr3 = roc_auc_score(testGen.classes, preds, multi_class='ovr', average='macro')
scr2 = roc_auc_score(testGen.classes, preds, multi_class='ovo', average='weighted')
scr4 = roc_auc_score(testGen.classes, preds, multi_class='ovr', average='weighted')


print("macro - ovo ROC AUC Score: ", scr1)
print("")

print("weighted - ovo ROC AUC Score: ", scr2)
print("")

print("macro - ovr ROC AUC Score: ", scr3)
print("")

print("weighted - ovr ROC AUC Score: ", scr4)
print("")

macro - ovo ROC AUC Score:  0.47161400754444716

weighted - ovo ROC AUC Score:  0.4767279306269024

macro - ovr ROC AUC Score:  0.4726454584590176

weighted - ovr ROC AUC Score:  0.48320816033806097



In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

target_names = ['A', 'F', 'PT', 'TA', 'DC','LC', 'MC', 'PC']
n_class = 8

for i in range(len(target_names)):
    fpr[i], tpr[i], thresh[i] = roc_curve(testGen.classes, predIdxs[:], pos_label=i)
    auroc = round(metrics.auc(fpr[i], tpr[i]),2)
    print('class',i,'-', target_names[i],' :','--AUC--->',auroc)

# plotting
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0-A: vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1- F: vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2- PT: vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--',color='yellow', label='Class 3- TA: vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='-',color='orange', label='Class 4-DC vs Rest')
plt.plot(fpr[5], tpr[5], linestyle='-',color='green', label='Class 5-LC: vs Rest')
plt.plot(fpr[6], tpr[6], linestyle='-',color='blue', label='Class 6-MC: vs Rest')
plt.plot(fpr[7], tpr[7], linestyle='-',color='yellow', label='Class 7- LC: vs Rest')


plt.title('Multiclass ROC-AUC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('8 class ROC ',dpi=300, bbox_inches='tight',   pad_inches = 0);


plt.show()

class 0 - A  : --AUC---> 0.5
class 1 - F  : --AUC---> 0.45
class 2 - PT  : --AUC---> 0.57
class 3 - TA  : --AUC---> 0.45
class 4 - DC  : --AUC---> 0.48
class 5 - LC  : --AUC---> 0.49
class 6 - MC  : --AUC---> 0.6
class 7 - PC  : --AUC---> 0.53


In [ ]:
import sklearn
acc = sklearn.metrics.accuracy_score(testGen.classes, predIdxs)
print(acc)

0.06716417910447761
